In [44]:
import os
os.chdir('D:\\Project\Dataset\Deep Learning\Fake News')

In [45]:
import pandas as pd

In [46]:
train_data = pd.read_csv('train.csv')

In [47]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [48]:
#Handling NaN values

train_data = train_data.dropna()

In [49]:
X = train_data.drop('label', axis = 1)
Y = train_data['label']

In [50]:
import tensorflow as tf

In [51]:
tf.__version__

'2.4.1'

In [52]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [53]:
#Vocabulary size

voc_size = 5000

### Onehot Representation

In [54]:
messages = X.copy()

In [55]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [56]:
messages.reset_index(inplace = True)  # Resetting index cos I had dropped NaN values earlier

In [57]:
import nltk
import re    #Regular Expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [58]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jisha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
# Dataset Preprocessing

PS = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    
    #Here I substitute everything except those that constitute of a-z or A-Z, with a ' ', in the title
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])  
    review = review.lower() #moving to lower ase
    review = review.split() 
    
    # Here I perform stemming only if a word in review is not present in the stopwords
    review = [PS.stem(word) for word in review if not word in stopwords.words('english')]
    # Combining all the words together to form a sentence
    review = ' '.join(review)
    # Appending it to corpus
    corpus.append(review)

In [60]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [61]:
# One hot representation gives each word an index with respect to the vocabulary size.
onehot_representation = [one_hot(words,voc_size) for words in corpus] 

In [62]:
onehot_representation

[[40, 3122, 3419, 4351, 3895, 4400, 2505, 569, 2266, 2407],
 [2724, 2100, 629, 2117, 1692, 684, 3465],
 [4318, 1635, 1725, 473],
 [3628, 4827, 4265, 2361, 1444, 3325],
 [1720, 1692, 2988, 18, 3715, 4133, 1692, 224, 818, 4256],
 [1802,
  3333,
  1886,
  3429,
  3169,
  1499,
  3080,
  1902,
  4052,
  244,
  1852,
  2186,
  1073,
  1413,
  3465],
 [4453, 3918, 3968, 2007, 570, 863, 2128, 1778, 584, 284, 112],
 [128, 1479, 4350, 1765, 520, 205, 1499, 3450, 584, 284, 112],
 [3907, 1044, 1142, 1167, 1351, 3295, 603, 2575, 1499, 2146],
 [1784, 4866, 761, 2068, 3248, 1272, 1442, 4136],
 [3977, 87, 3038, 4352, 3289, 1196, 1208, 2779, 4946, 2586, 3610],
 [2361, 4683, 3895, 3295, 1499, 520],
 [3722, 4918, 2992, 3425, 2746, 1136, 1398, 243, 2575],
 [4726, 1797, 4211, 4844, 2004, 2916, 1035, 584, 284, 112],
 [1098, 4971, 3804, 2124, 4295, 584, 284, 112],
 [2956, 965, 255, 2867, 4069, 1622, 2834, 2500, 1891, 3161],
 [1407, 1778, 2100],
 [3158, 1094, 2722, 4101, 1499, 557, 788, 3465],
 [350, 3288, 6

### Embedding Representation

In [63]:
sent_length = 20

# I am using Pad Sequencing to have all my sentences of the same length by padding at prefix.
embedded_docs = pad_sequences(onehot_representation, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ...  569 2266 2407]
 [   0    0    0 ... 1692  684 3465]
 [   0    0    0 ... 1635 1725  473]
 ...
 [   0    0    0 ...  584  284  112]
 [   0    0    0 ... 4008 2980 1594]
 [   0    0    0 ... 2557 2729 3995]]


In [64]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   40,
       3122, 3419, 4351, 3895, 4400, 2505,  569, 2266, 2407])

### Model Creation

In [65]:
embedding_vector_features = 40

model = Sequential()

#Embedding Layer
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
# One LSTM layer with 100 neurons
model.add(LSTM(100))
# Sigmoid in Dense for Classification
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [66]:
len(embedded_docs),Y.shape

(18285, (18285,))

In [67]:
import numpy as np

X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [68]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

### Model Training

In [70]:
model.fit(X_train, Y_train, validation_split = 0.3, epochs = 30, batch_size = 64)

Epoch 1/30
134/134 [==============================] - 4s 16ms/step - loss: 0.5276 - accuracy: 0.7536 - val_loss: 0.2103 - val_accuracy: 0.9118
Epoch 2/30
134/134 [==============================] - 2s 12ms/step - loss: 0.1515 - accuracy: 0.9422 - val_loss: 0.2034 - val_accuracy: 0.9127
Epoch 3/30
134/134 [==============================] - 2s 12ms/step - loss: 0.0885 - accuracy: 0.9698 - val_loss: 0.2136 - val_accuracy: 0.9238
Epoch 4/30
134/134 [==============================] - 2s 13ms/step - loss: 0.0570 - accuracy: 0.9819 - val_loss: 0.2220 - val_accuracy: 0.9167
Epoch 5/30
134/134 [==============================] - 2s 14ms/step - loss: 0.0408 - accuracy: 0.9900 - val_loss: 0.3155 - val_accuracy: 0.9075
Epoch 6/30
134/134 [==============================] - 2s 15ms/step - loss: 0.0262 - accuracy: 0.9927 - val_loss: 0.3333 - val_accuracy: 0.9181
Epoch 7/30
134/134 [==============================] - 2s 14ms/step - loss: 0.0156 - accuracy: 0.9969 - val_loss: 0.3800 - val_accuracy: 0.9127

### Performance Metrics And Accuracy

In [71]:
Y_pred = (model.predict(X_test) > 0.5).astype("int32")

In [72]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, Y_pred)

array([[3061,  358],
       [ 265, 2351]], dtype=int64)

In [73]:
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, Y_pred)

0.8967688483844242

### Adding Dropout

In [75]:
from tensorflow.keras.layers import Dropout

# Creating model

embedding_vector_features = 40

model=Sequential()

model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [76]:
model.fit(X_train, Y_train, validation_split = 0.3, epochs = 30, batch_size = 64)

Epoch 1/30
134/134 [==============================] - 4s 18ms/step - loss: 0.5528 - accuracy: 0.6870 - val_loss: 0.2206 - val_accuracy: 0.9067
Epoch 2/30
134/134 [==============================] - 2s 16ms/step - loss: 0.1668 - accuracy: 0.9356 - val_loss: 0.2058 - val_accuracy: 0.9173
Epoch 3/30
134/134 [==============================] - 2s 15ms/step - loss: 0.1095 - accuracy: 0.9599 - val_loss: 0.2119 - val_accuracy: 0.9235
Epoch 4/30
134/134 [==============================] - 2s 15ms/step - loss: 0.0779 - accuracy: 0.9746 - val_loss: 0.2152 - val_accuracy: 0.9238
Epoch 5/30
134/134 [==============================] - 2s 15ms/step - loss: 0.0618 - accuracy: 0.9784 - val_loss: 0.2597 - val_accuracy: 0.9227
Epoch 6/30
134/134 [==============================] - 2s 15ms/step - loss: 0.0412 - accuracy: 0.9882 - val_loss: 0.2837 - val_accuracy: 0.9181
Epoch 7/30
134/134 [==============================] - 2s 15ms/step - loss: 0.0387 - accuracy: 0.9890 - val_loss: 0.3475 - val_accuracy: 0.9195

In [77]:
Y_pred = (model.predict(X_test) > 0.5).astype('int32')

In [78]:
confusion_matrix(Y_test, Y_pred)

array([[3140,  279],
       [ 361, 2255]], dtype=int64)

In [79]:
accuracy_score(Y_test, Y_pred)

0.8939519469759735